# Testing Notebook for Keysight 81150A AWG Driver

This notebook is designed to walk a technician through testing the functionality of the `Keysight81150a` driver. 

**Instructions for the Technician:**
1. Ensure a Keysight 81150A Arbitrary Waveform Generator is connected to the same network as this computer.
2. Connect the output of Channel 1 of the AWG to an oscilloscope to visualize the waveforms.
3. Run each cell sequentially.
4. After running a cell that sends a command to the instrument, observe the AWG's screen and the connected oscilloscope to verify that the expected change has occurred.
5. For cells that query the instrument, check the output in the notebook to ensure it is reasonable.

## 1. Connect to the Instrument

In [ ]:
import numpy as np
from piec.drivers.Keysight.k_81150a import Keysight81150a
from piec.drivers.utilities import PiecManager

# Initialize PiecManager and list available resources
pm = PiecManager()
pm.list_resources()

**Technician Note:** Find the VISA address for the Keysight AWG in the list above and replace `'Address'` with the correct string (e.g., `'TCPIP0::A-81150A-00000.local::inst0::INSTR'`).

In [ ]:
# Connect to the instrument
# The technician will replace 'Address' with the actual instrument address from the list above.
awg = Keysight81150a("Address")

## 2. Standard Waveform Configuration Tests

In [ ]:
# Configure a Sine Wave on Channel 1
awg.configure_waveform(
    channel=1,
    waveform='SIN',
    frequency=1e6,  # 1 MHz
    amplitude=2.0,  # 2 Vpp
    offset=0.5      # 0.5 V
)

**Expected Output:** The AWG screen for Channel 1 should show:
- **Waveform:** Sine
- **Frequency:** 1.000 MHz
- **Amplitude:** 2.000 Vpp
- **Offset:** 500.000 mV

The oscilloscope should display a 1 MHz sine wave with a 2V peak-to-peak amplitude and a 0.5V DC offset.

In [ ]:
# Turn on Channel 1 Output
awg.output(channel=1, on=True)

**Expected Output:** The 'CH 1' output button on the AWG front panel should illuminate. The sine wave should now be visible on the oscilloscope.

In [ ]:
# Configure a Square Wave on Channel 1
awg.set_waveform(channel=1, waveform='SQU')
awg.set_square_duty_cycle(channel=1, duty_cycle=25)

**Expected Output:** The AWG screen for Channel 1 should now show a Square wave with a 25% duty cycle. The oscilloscope display will update to show the new waveform.

In [ ]:
# Configure a Ramp Wave on Channel 1
awg.set_waveform(channel=1, waveform='RAMP')
awg.set_ramp_symmetry(channel=1, symmetry=80)

**Expected Output:** The AWG screen for Channel 1 should now show a Ramp wave with 80% symmetry (a fast rise and slow fall). The oscilloscope display will update.

In [ ]:
# Turn off Channel 1 Output
awg.output(channel=1, on=False)

**Expected Output:** The 'CH 1' output button light should turn off. The waveform should disappear from the oscilloscope.

## 3. Pulse Configuration Test

In [ ]:
# Configure a Pulse on Channel 1
awg.configure_pulse(
    channel=1,
    pulse_width=1e-6, # 1 µs
    rise_time=10e-9,  # 10 ns
    fall_time=10e-9,  # 10 ns
    pulse_delay=0.5e-6 # 0.5 µs
)
# Also set frequency and amplitude
awg.set_frequency(channel=1, frequency=500e3) # 500 kHz
awg.set_amplitude(channel=1, amplitude=3.0)  # 3 Vpp

**Expected Output:** The AWG screen for Channel 1 should be set to a Pulse waveform with the specified width, transition times, and delay. The frequency should be 500 kHz and amplitude 3 Vpp. Turn on the output to verify on the oscilloscope.

In [ ]:
awg.output(channel=1, on=True)

## 4. Arbitrary Waveform Test

In [ ]:
# Create a custom waveform: a sine wave with a glitch
# Note: The data points are scaled internally by the driver to the instrument's required integer format.
points = 200
x = np.linspace(0, 2 * np.pi, points)
custom_data = np.sin(x) * 0.8 # Create a sine wave at 80% of full scale
custom_data[50:60] = 1.0 # Add a glitch at full positive scale
custom_data[150:160] = -1.0 # Add a glitch at full negative scale

# Send the waveform to the instrument's volatile memory
awg.create_arb_waveform(channel=1, name='GLITCH', data=custom_data)

**Expected Output:** A message `Estimated Peak-to-Peak Ratio...` should be printed. On the instrument, a new arbitrary waveform named `GLITCH` should be created in the waveform catalog. There may be a short delay as the data is transferred.

In [ ]:
# Set the AWG to output the created arbitrary waveform
awg.set_arb_waveform(channel=1, name='GLITCH')
awg.set_frequency(channel=1, frequency=100e3) # Set a frequency for the arb waveform

**Expected Output:** The AWG's function for Channel 1 should now be `USER` and the selected user waveform should be `GLITCH`. The oscilloscope should display the sine wave with the positive and negative glitches.

## 5. Amplifier and Trigger Tests

In [ ]:
# Set the amplifier to High Voltage mode
awg.configure_output_amplifier(channel=1, type='HIV')

**Expected Output:** The AWG should switch to High Voltage output mode. This allows for higher voltage swings (up to 10 Vpp into 50 Ohm), but may limit the maximum frequency for some waveforms. This change might be indicated on the instrument's display.

In [ ]:
# Set a high amplitude to test the new range
awg.set_amplitude(channel=1, amplitude=8.0) # 8 Vpp

**Expected Output:** The amplitude should be set to 8.0 Vpp, which is only possible in 'HIV' mode. The oscilloscope should show a larger waveform.

In [ ]:
# Configure the trigger
awg.configure_trigger(channel=1, trigger_source='EXT', trigger_slope='POS', trigger_mode='EDGE')

**Expected Output:** The AWG's trigger settings for Channel 1 should be updated:
- **Source:** Ext
- **Slope:** Positive (Rising)
- **Mode:** Edge

The AWG will now wait for an external trigger signal to generate a waveform (if in a triggered mode).

## 6. End of Test

If all the steps above were completed and the observed behavior on the AWG and oscilloscope matched the expected output, the driver is functioning correctly.